In [29]:
import torch
import torchvision
from torch import nn
from torchvision import transforms
from MLProject2Dataset import *
from CNNModels import *

In [30]:
# transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
# batch_size = 4

# trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

# testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
# testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

# ΑΠΛΟ ΣΝΔ

- Input             3 * 50 * 62 = 9300
- 1st conv          32 * 48 * 60 = 92160
- 1st maxpool       32 * 24 * 30 = 23040
- 2nd conv          64 * 22 * 28 = 39424
- 2nd maxpool       64 * 11 * 14 = 9856
- 3rd conv          64 * 9 * 12 = 6912
- 3rd maxpool       64 * 4 * 6 = 1536


### Train

In [31]:
dataset = MLProject2Dataset('./data/dermoscopy_classification', transform=image_transforms(50, 62))
# dataset = MLProject2DatasetSmall('./data/dermoscopy_classification', transform=image_transforms(50, 62))

generator = torch.Generator().manual_seed(42)
train, validate, test = torch.utils.data.random_split(dataset, [0.6, 0.1, 0.3], generator)

In [32]:
len(dataset)

10015

In [33]:
train_dataloader = DataLoader(train, batch_size=64, shuffle=True)
validate_dataloader = DataLoader(validate, batch_size=64, shuffle=False)
test_dataloader = DataLoader(test, batch_size=64, shuffle=False)

In [34]:
device = ("cuda" if torch.cuda.is_available()
          else "mps" if torch.backends.mps.is_available()
          else "cpu")

model = SimpleCNN().to(device)
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001)

In [35]:
train_fn(model, train_dataloader, validate_dataloader, optimizer = optimizer, loss=loss_fn, device=device, print_period=10)

Training on cuda
Epoch: 0, batch:     5] train loss: 1.954 train acc: 0.131 | val loss: 1.953 val acc: 0.128
Epoch: 0, batch:    10] train loss: 1.954 train acc: 0.128 | val loss: 1.953 val acc: 0.128
Epoch: 0, batch:    15] train loss: 1.954 train acc: 0.127 | val loss: 1.952 val acc: 0.128
Epoch: 0, batch:    20] train loss: 1.952 train acc: 0.129 | val loss: 1.951 val acc: 0.129
Epoch: 0, batch:    25] train loss: 1.951 train acc: 0.132 | val loss: 1.951 val acc: 0.130


KeyboardInterrupt: 